## Preparation 

In [ ]:
!pip uninstall spacy

In [12]:
import nltk
import spacy
spacy.load('en_core_web_sm') # run once，the model can be changed according to the real requirements

ModuleNotFoundError: No module named 'spacy'

In [2]:
import pandas as pd
import numpy as np
import re
import gensim
from vaderSentiment import vaderSentiment

## Loading Data

In [3]:
path = os.path.dirname(os.path.abspath('CLV-Final.py'))
df = pd.read_json(path +"/Sports_and_Outdoors_5.json", lines=True)
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1881509818,"[0, 0]",5,This came in on time and I am veru happy with ...,"01 26, 2014",AIXZKN4ACSKI,David Briner,Woks very good,1390694400
1,1881509818,"[1, 1]",5,I had a factory Glock tool that I was using fo...,"02 2, 2012",A1L5P841VIO02V,Jason A. Kramer,Works as well as the factory tool,1328140800
2,1881509818,"[2, 2]",4,If you don't have a 3/32 punch or would like t...,"02 28, 2012",AB2W04NI4OEAD,J. Fernald,"It's a punch, that's all.",1330387200
3,1881509818,"[0, 0]",4,This works no better than any 3/32 punch you w...,"02 5, 2012",A148SVSWKTJKU6,"Jusitn A. Watts ""Maverick9614""",It's a punch with a Glock logo.,1328400000
4,1881509818,"[0, 0]",4,I purchased this thinking maybe I need a speci...,"04 23, 2013",AAAWJ6LW9WMOO,Material Man,"Ok,tool does what a regular punch does.",1366675200


In [8]:
Train = df.loc[:,['asin','overall','reviewText','reviewTime','reviewerName']]

In [9]:
Train['reviewTime'] = pd.to_datetime(Train['reviewTime'])
Train.sort_values('reviewTime', ascending=False,inplace=True)

In [10]:
Train = Train.reset_index()
Train = Train.drop(['index'],axis=1)
Train

,asin,overall,reviewText,reviewTime,reviewerName
0,B00LFPS0CY,5,I have never been so impressed by a water bott...,2014-07-23,"Dianne E. Socci-Tetro ""Never Judge a Book by ..."
1,B00GOGV314,5,"I have a large wrist, and the large fits just ...",2014-07-23,Spencer M
2,B007Q4NCZI,5,easy and quick to clean the chain,2014-07-23,NaN
3,B001B6NEZU,5,very efficient and fast,2014-07-23,NaN
4,B004269CM0,5,easy to use turning the wrench anyway you wanted,2014-07-23,NaN
5,B000IZEH1K,5,works well and I recommend others to try it,2014-07-23,NaN
6,B00D9B7OKQ,5,very durable and handy to clean and fix the bike,2014-07-23,NaN
7,B003EMP89Y,3,Bottle opener doesn't work that good.,2014-07-23,NaN
8,B00CDI7TRY,2,So I want to start off by saying that had the ...,2014-07-22,Amazon Customer
9,B00E1IDFYU,5,Great handlebars for the price. They feel lig...,2014-07-22,Justin L.


In [11]:
Train = Train.loc[0:1999]
Train['asin'].describe()

count           2000
unique          1506
top       B00L1N7FDK
freq              19
Name: asin, dtype: object

,asin,overall,reviewText,reviewTime,reviewerName,processedText,review_sentiment
0,B00LFPS0CY,5,I have never been so impressed by a water bott...,2014-07-23,"Dianne E. Socci-Tetro ""Never Judge a Book by ...","-PRON- impress water bottle , -PRON- one . -P...",0.9833
1,B00GOGV314,5,"I have a large wrist, and the large fits just ...",2014-07-23,Spencer M,"-PRON- large wrist , large fit snugly comfort...",0.9360
2,B007Q4NCZI,5,easy and quick to clean the chain,2014-07-23,NaN,easy quick clean chain,0.6808
3,B001B6NEZU,5,very efficient and fast,2014-07-23,NaN,efficient fast,0.4215
4,B004269CM0,5,easy to use turning the wrench anyway you wanted,2014-07-23,NaN,easy use turn wrench want,0.4939
5,B000IZEH1K,5,works well and I recommend others to try it,2014-07-23,NaN,work -PRON- recommend try,0.3612
6,B00D9B7OKQ,5,very durable and handy to clean and fix the bike,2014-07-23,NaN,durable handy clean fix bike,0.4019
7,B003EMP89Y,3,Bottle opener doesn't work that good.,2014-07-23,NaN,bottle opener do not work good .,-0.3412
8,B00CDI7TRY,2,So I want to start off by saying that had the ...,2014-07-22,Amazon Customer,so -PRON- want start say quality better easil...,0.2398
9,B00E1IDFYU,5,Great handlebars for the price. They feel lig...,2014-07-22,Justin L.,"great handlebar price . -PRON- feel light , s...",0.9231


## Clean Data

Removing stopwords has three libraries:NLTK.spaCy,Gensim. The choice of each method will influence the future step of normalization. Here we use spaCy for the whole data pro-processing to ensure a unified criteria. However, other methods are also applicable for the cleaning data process. Besides, spaCy can realise functions such as Neural network models,Integrated word vectors and Dependency parsing. Consider the future usage, we use spaCy at the beginning.

To pre-process the data, here are some steps needs to be finished:
1.Regularize the text.
2.Remove the too short words that has little meanings
3.Lower all words 
4.Remove stop words
5.Tokenized words
6.Stem

The step 5 and step 6 can realise in spaCy. 

In [12]:
# remove stop words
stop_words = spacy.lang.en.stop_words.STOP_WORDS
Train['processedText'] = Train['reviewText'].apply(lambda x: ' '.join(w for w in x.split() if not w in stop_words ))

# remove unwanted characters, numbers and symbols
Train['processedText'] = Train['processedText'].str.replace("[^a-zA-Z#]", " ")

# remove short words (length < 3)
Train['processedText'] = Train['processedText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# make entire text lowercase
Train['processedText'] = Train['processedText'].str.lower()

Train


C:\Users\yaohu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\yaohu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yaohu\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,asin,overall,reviewText,reviewTime,reviewerName,processedText
0,B00LFPS0CY,5,I have never been so impressed by a water bott...,2014-07-23,"Dianne E. Socci-Tetro ""Never Judge a Book by ...",impressed water bottle mean bottle bottle righ...
1,B00GOGV314,5,"I have a large wrist, and the large fits just ...",2014-07-23,Spencer M,large wrist large fits snugly comfortable purc...
2,B007Q4NCZI,5,easy and quick to clean the chain,2014-07-23,NaN,easy quick clean chain
3,B001B6NEZU,5,very efficient and fast,2014-07-23,NaN,efficient fast
4,B004269CM0,5,easy to use turning the wrench anyway you wanted,2014-07-23,NaN,easy turning wrench wanted
5,B000IZEH1K,5,works well and I recommend others to try it,2014-07-23,NaN,works recommend
6,B00D9B7OKQ,5,very durable and handy to clean and fix the bike,2014-07-23,NaN,durable handy clean bike
7,B003EMP89Y,3,Bottle opener doesn't work that good.,2014-07-23,NaN,bottle opener doesn work good
8,B00CDI7TRY,2,So I want to start off by saying that had the ...,2014-07-22,Amazon Customer,want start saying quality better easily reache...
9,B00E1IDFYU,5,Great handlebars for the price. They feel lig...,2014-07-22,Justin L.,great handlebars price they feel light solid r...


## Sentimental Analysis
NLP process can help read text in different pipelines. The three main pipelines are : tagger for Assign part-of-speech tags, parser for Assign dependency labels and ner for Detect and label named entities. Here we ignore parser to save process time.



In [13]:
nlp = spacy.load('en_core_web_sm')
review_sentiment = []
lemma = []
Total = []
NamedEntity = []
analyzer = vaderSentiment.SentimentIntensityAnalyzer()
for w in Train['processedText']:
    sentences = ''
    doc = nlp(w)
    for token in doc:
        sentences= sentences+' '+token.lemma_
    sentiment = analyzer.polarity_scores(sentences)
    lemma.append(sentences)
    Total.append(sentiment)
    review_sentiment.append(sentiment['compound'])
    
   

In [14]:
Train['review_sentiment'] = review_sentiment
Train['Sentiment'] = 'Negative'
Train.loc[Train['review_sentiment']>-0.5,'Sentiment'] = 'Neutral' 
Train.loc[Train['review_sentiment']>0.5,'Sentiment'] = 'Positive'
Train

C:\Users\yaohu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\yaohu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yaohu\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

,asin,overall,reviewText,reviewTime,reviewerName,processedText,review_sentiment,Sentiment
0,B00LFPS0CY,5,I have never been so impressed by a water bott...,2014-07-23,"Dianne E. Socci-Tetro ""Never Judge a Book by ...",impressed water bottle mean bottle bottle righ...,0.9802,Positive
1,B00GOGV314,5,"I have a large wrist, and the large fits just ...",2014-07-23,Spencer M,large wrist large fits snugly comfortable purc...,0.9360,Positive
2,B007Q4NCZI,5,easy and quick to clean the chain,2014-07-23,NaN,easy quick clean chain,0.6808,Positive
3,B001B6NEZU,5,very efficient and fast,2014-07-23,NaN,efficient fast,0.4215,Neutral
4,B004269CM0,5,easy to use turning the wrench anyway you wanted,2014-07-23,NaN,easy turning wrench wanted,0.4939,Neutral
5,B000IZEH1K,5,works well and I recommend others to try it,2014-07-23,NaN,works recommend,0.3612,Neutral
6,B00D9B7OKQ,5,very durable and handy to clean and fix the bike,2014-07-23,NaN,durable handy clean bike,0.4019,Neutral
7,B003EMP89Y,3,Bottle opener doesn't work that good.,2014-07-23,NaN,bottle opener doesn work good,0.4404,Neutral
8,B00CDI7TRY,2,So I want to start off by saying that had the ...,2014-07-22,Amazon Customer,want start saying quality better easily reache...,0.7096,Positive
9,B00E1IDFYU,5,Great handlebars for the price. They feel lig...,2014-07-22,Justin L.,great handlebars price they feel light solid r...,0.9231,Positive


## Keywords Extraction

In [15]:
from sklearn.feature_extraction.text import  TfidfVectorizer
Tf_Idf = TfidfVectorizer(decode_error='replace',ngram_range=(1,2),encoding='utf-8')
Tf_Idf_score = Tf_Idf.fit_transform(Train['processedText']) 
weight = Tf_Idf_score.toarray()
feature_names = Tf_Idf.get_feature_names() 

In [16]:
vocab = ()
sklearn_dict ={}
keywords =[]
Dic_key = {}
Train_keywords = []
for i in range(len(Train['processedText'])):
    for w in Train['processedText'].loc[i].split():
        vocab = Tf_Idf.vocabulary_.get(w)
        sklearn_dict[w] = weight[i][vocab]
        Dic_key = sorted(sklearn_dict.items(),key=lambda x: x[1].all(), reverse=True)[:5]
    for k,(w,n) in enumerate(Dic_key):
        keywords.append(w)
    Train_keywords.append(keywords)
    keywords = []
    sklearn_dict = {}

In [17]:
Train['keywords'] = Train_keywords
Train

C:\Users\yaohu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,asin,overall,reviewText,reviewTime,reviewerName,processedText,review_sentiment,Sentiment,keywords
0,B00LFPS0CY,5,I have never been so impressed by a water bott...,2014-07-23,"Dianne E. Socci-Tetro ""Never Judge a Book by ...",impressed water bottle mean bottle bottle righ...,0.9802,Positive,"[impressed, water, bottle, mean, right]"
1,B00GOGV314,5,"I have a large wrist, and the large fits just ...",2014-07-23,Spencer M,large wrist large fits snugly comfortable purc...,0.9360,Positive,"[large, wrist, fits, snugly, comfortable]"
2,B007Q4NCZI,5,easy and quick to clean the chain,2014-07-23,NaN,easy quick clean chain,0.6808,Positive,"[easy, quick, clean, chain]"
3,B001B6NEZU,5,very efficient and fast,2014-07-23,NaN,efficient fast,0.4215,Neutral,"[efficient, fast]"
4,B004269CM0,5,easy to use turning the wrench anyway you wanted,2014-07-23,NaN,easy turning wrench wanted,0.4939,Neutral,"[easy, turning, wrench, wanted]"
5,B000IZEH1K,5,works well and I recommend others to try it,2014-07-23,NaN,works recommend,0.3612,Neutral,"[works, recommend]"
6,B00D9B7OKQ,5,very durable and handy to clean and fix the bike,2014-07-23,NaN,durable handy clean bike,0.4019,Neutral,"[durable, handy, clean, bike]"
7,B003EMP89Y,3,Bottle opener doesn't work that good.,2014-07-23,NaN,bottle opener doesn work good,0.4404,Neutral,"[bottle, opener, doesn, work, good]"
8,B00CDI7TRY,2,So I want to start off by saying that had the ...,2014-07-22,Amazon Customer,want start saying quality better easily reache...,0.7096,Positive,"[want, start, saying, quality, better]"
9,B00E1IDFYU,5,Great handlebars for the price. They feel lig...,2014-07-22,Justin L.,great handlebars price they feel light solid r...,0.9231,Positive,"[great, handlebars, price, they, feel]"


## Topic Modelling

In [18]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
# Convert to list data
data = Train['keywords'].values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
data_words

[['impressed', 'water', 'bottle', 'mean', 'right'],
 ['large', 'wrist', 'fits', 'snugly', 'comfortable'],
 ['easy', 'quick', 'clean', 'chain'],
 ['efficient', 'fast'],
 ['easy', 'turning', 'wrench', 'wanted'],
 ['works', 'recommend'],
 ['durable', 'handy', 'clean', 'bike'],
 ['bottle', 'opener', 'doesn', 'work', 'good'],
 ['want', 'start', 'saying', 'quality', 'better'],
 ['great', 'handlebars', 'price', 'they', 'feel'],
 ['awsome'],
 ['heard', 'monkey', 'fist', 'looked', 'history'],
 ['works', 'great', 'ships', 'fast'],
 ['great', 'scope', 'completely', 'unusable', 'closer'],
 ['cheap', 'scope', 'serves', 'purpose'],
 ['finally', 'good', 'water', 'bottle', 'break'],
 ['kids', 'love', 'bobble', 'water', 'bottles'],
 ['these', 'grill', 'mats', 'grilling', 'easier'],
 ['love', 'fact', 'fruit', 'water', 'tasting'],
 ['great', 'product'],
 ['very', 'small', 'bench', 'wife', 'frame'],
 ['very', 'accurate', 'google', 'earth', 'determine'],
 ['have', 'item', 'light', 'friends', 'tell'],
 ['st

In [23]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [24]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [25]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Initialize spacy 'en_core_web_sm' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [26]:
from gensim import corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]




In [37]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [39]:
NUM_TOPICS = 9
word_dict = {};
for i in range(NUM_TOPICS):
    words = lda_model.show_topic(i, topn = 20)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09
0,great,look,quality,good,love,product,bottle,hold,nice
1,work,awesome,clean,buy,very,light,water,easy,price
2,knife,pack,take,pretty,come,bright,stuff,review,little
3,perfect,one,bike,tool,make,super,sight,install,need
4,excellent,color,handy,case,small,advertise,keep,decent,size
5,comfortable,safe,high,right,purchase,think,shoot,glock,cheap
6,well,different,say,really,sure,expect,quickly,camping,money
7,time,diamond,want,wear,easy,exactly,year,feel,item
8,fit,starter,start,break,thing,value,cleaning,read,get
9,mount,barrel,deal,home,grip,try,minute,piece,go


In [42]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data_lemmatized):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_lemmatized)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)
   

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.3285,"quality, clean, take, bike, handy, high, say, ...","[impressed, water, bottle, mean]"
1,1,0.0,0.7104,"great, work, knife, perfect, excellent, comfor...","[large, wrist, fit, snugly, comfortable]"
2,2,0.0,0.3863,"great, work, knife, perfect, excellent, comfor...","[easy, quick, clean, chain]"
3,3,0.0,0.5288,"great, work, knife, perfect, excellent, comfor...","[efficient, fast]"
4,4,7.0,0.3692,"hold, easy, review, install, decent, glock, ca...","[easy, turn, want]"
5,5,4.0,0.3475,"love, very, come, make, small, purchase, sure,...",[recommend]
6,6,2.0,0.6186,"quality, clean, take, bike, handy, high, say, ...","[durable, handy, clean, bike]"
7,7,2.0,0.3268,"quality, clean, take, bike, handy, high, say, ...","[opener, doesn, work, good]"
8,8,2.0,0.6669,"quality, clean, take, bike, handy, high, say, ...","[want, start, say, quality, better]"
9,9,0.0,0.2822,"great, work, knife, perfect, excellent, comfor...","[great, price, feel]"


In [43]:
df_dominant_topic.to_csv(r'C:\Users\yaohu\Desktop\TopicModelResult.csv')

## Evaluate the topic numbers

In [38]:
# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.6681785788518846


## VIsualize the Topic

In [41]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word,sort_topics=False)
vis 



C:\Users\yaohu\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.381872 -0.003787       1        1  22.112717
1     -0.037381 -0.008562       2        1   5.302793
2     -0.045402 -0.018841       3        1   9.210676
3     -0.053093 -0.087417       4        1  13.359300
4     -0.064961 -0.142841       5        1  12.023326
5     -0.042326 -0.012406       6        1   9.540859
6     -0.040245 -0.012246       7        1   7.025552
7     -0.047673 -0.043548       8        1   7.070487
8     -0.050792  0.329649       9        1  14.354290, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
30    Default  290.000000        great  290.000000  30.0000  30.0000
22    Default  177.000000         good  177.000000  29.0000  29.0000
24    Default  194.000000         work  194.000000  28.0000  28.0000
48    Default   86.000000         love   86.000000  27.0000  27.0000
82    Default   88.000000          buy   88.000000  26.0000  26.0000
54    Default   66.000000      product   66.000000  25.0000  25.0000
183   Default   78.000000         nice   78.000000  24.0000  24.0000
58    Default   70.000000         very   70.000000  23.0000  23.0000
26    Default   56.000000      quality   56.000000  22.0000  22.0000
31    Default   68.000000        price   68.000000  21.0000  21.0000
125   Default   82.000000        knife   82.000000  20.0000  20.0000
193   Default   54.000000       little   54.000000  19.0000  19.0000
35    Default   33.000000         look   33.000000  18.0000  18.0000
64    Default   41.000000        light   41.000000  17.0000  17.0000
254   Default   51.000000         need   51.000000  16.0000  16.0000
139   Default   67.000000      perfect   67.000000  15.0000  15.0000
72    Default   44.000000         come   44.000000  14.0000  14.0000
379   Default   39.000000         make   39.000000  13.0000  13.0000
11    Default   42.000000         easy   42.000000  12.0000  12.0000
57    Default   37.000000        small   37.000000  11.0000  11.0000
116   Default   28.000000         hold   28.000000  10.0000  10.0000
197   Default   47.000000    excellent   47.000000   9.0000   9.0000
0     Default   24.000000       bottle   24.000000   8.0000   8.0000
10    Default   27.000000        clean   27.000000   7.0000   7.0000
4     Default   46.000000  comfortable   46.000000   6.0000   6.0000
206   Default   34.000000         size   34.000000   5.0000   5.0000
41    Default   29.000000        cheap   29.000000   4.0000   4.0000
3     Default   19.000000        water   19.000000   3.0000   3.0000
252   Default   37.000000         well   37.000000   2.0000   2.0000
176   Default   37.000000         time   37.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
183    Topic9   77.390938         nice   78.082787   1.9322  -2.6389
31     Topic9   67.854813        price   68.546631   1.9310  -2.7704
193    Topic9   54.153534       little   54.845364   1.9284  -2.9960
254    Topic9   50.789047         need   51.480911   1.9276  -3.0601
206    Topic9   33.657398         size   34.349255   1.9208  -3.4716
41     Topic9   29.167665        cheap   29.859505   1.9177  -3.6147
271    Topic9   25.703962        money   26.395792   1.9146  -3.7411
63     Topic9   25.101118         item   25.792938   1.9139  -3.7649
241    Topic9   24.030069          get   24.722122   1.9127  -3.8085
344    Topic9   23.167719           go   23.859579   1.9117  -3.8450
343    Topic9   16.796827         find   17.488611   1.9008  -4.1666
39     Topic9   16.234419        scope   16.926226   1.8994  -4.2007
288    Topic9   15.580880      receive   16.272720   1.8977  -4.2417
150    Topic9   12.609420         cool   13.301263   1.8877  -4.4533
356    Topic9   12.470222        spend   13.162254   1.8871  -4.4644
345    Topic9   12.183468         hard   12.875251 